# 🗃️ Week 9 — Introduction to Databases and SQL

## 🎯 Session Overview

Welcome to **Databases and SQL**! This is where you'll learn to persist data beyond your program's runtime. Databases are the backbone of nearly every application - from simple to-do apps to massive social networks.

### What is a Database?

```
┌─────────────────────────────────────────────────────────────────┐
│                    DATABASE ARCHITECTURE                        │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   Python Code    ←──SQL Queries──→    SQLite Database          │
│   ┌─────────┐                         ┌─────────────────┐      │
│   │ sqlite3 │     SELECT * FROM...    │   📁 Tables     │      │
│   │ module  │ ──────────────────────→ │   ├─ users      │      │
│   │         │                         │   ├─ products   │      │
│   │         │ ←────────────────────── │   └─ orders     │      │
│   └─────────┘     [Results/Rows]      └─────────────────┘      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### Why Databases?

| Problem | Solution |
|---------|----------|
| 📁 **Files are messy** | Structured tables with defined columns |
| 🔍 **Hard to search** | SQL queries to find exactly what you need |
| 👥 **Multiple users** | Concurrent access with transactions |
| 🔒 **Data integrity** | Constraints, foreign keys, validation |
| 📊 **Big data** | Efficient storage and indexing |

### Database Types

| Type | Example | Best For |
|------|---------|----------|
| **Relational** | SQLite, PostgreSQL | Structured data with relationships |
| **Document** | MongoDB | Flexible, JSON-like data |
| **Key-Value** | Redis | Caching, simple lookups |
| **Graph** | Neo4j | Highly connected data |

*Today we'll focus on **SQLite** - a lightweight relational database built into Python!*

## 🎯 Learning Objectives

By the end of this session, you will be able to:

- ✅ **Create** SQLite databases using Python's `sqlite3` module
- ✅ **Query** data using SQL SELECT statements
- ✅ **Modify** data with INSERT, UPDATE, and DELETE
- ✅ **Protect** against SQL injection with parameterized queries
- ✅ **Design** basic database schemas with relationships
- ✅ **Apply** best practices for data integrity

### 📚 Topics Covered

| Section | Topic | Key Concept |
|---------|-------|-------------|
| 1 | Database Basics | Connections, cursors |
| 2 | Creating Tables | Schema design, data types |
| 3 | CRUD Operations | Create, Read, Update, Delete |
| 4 | Query Filters | WHERE, ORDER BY, LIMIT |
| 5 | SQL Injection | Parameterized queries |
| 6 | Best Practices | Context managers, transactions |

---

### 🔄 JavaScript Transition Note

If you're coming from JavaScript/web development:

| JavaScript | Python | Purpose |
|------------|--------|---------|
| `mysql2` / `pg` | `sqlite3` | Database driver |
| `Sequelize` / `Prisma` | SQLAlchemy | ORM |
| `await query()` | `cursor.execute()` | Run SQL |
| `results.rows` | `cursor.fetchall()` | Get results |

**Key Differences:**
- Python's `sqlite3` is **synchronous** (no async/await needed)
- SQLite is **file-based** (or in-memory) - no server required!
- Both use **parameterized queries** (`?` placeholders) for security

---

## 🔗 Section 1: Database Connections

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 DATABASE CONCEPTS OVERVIEW
# ═══════════════════════════════════════════════════════════════════════════════

print("""
🗃️ DATABASE KEY CONCEPTS
═══════════════════════════════════════════════════════════════════════════════

┌─────────────────────────────────────────────────────────────────────────────┐
│ RELATIONAL DATABASE STRUCTURE                                               │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│   DATABASE: my_app.db                                                       │
│   ├── TABLE: users                                                          │
│   │   ├── COLUMN: id (INTEGER, PRIMARY KEY)                                │
│   │   ├── COLUMN: name (TEXT)                                              │
│   │   └── COLUMN: email (TEXT, UNIQUE)                                     │
│   │                                                                         │
│   │   ROW: (1, 'Alice', 'alice@example.com')                               │
│   │   ROW: (2, 'Bob', 'bob@example.com')                                   │
│   │                                                                         │
│   └── TABLE: orders                                                         │
│       ├── COLUMN: id (INTEGER, PRIMARY KEY)                                │
│       ├── COLUMN: user_id (INTEGER, FOREIGN KEY → users.id)               │
│       └── COLUMN: total (REAL)                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

📝 KEY TERMINOLOGY:
  • TABLE = Like a spreadsheet (has columns and rows)
  • ROW = One record (also called a "tuple")
  • COLUMN = One field (also called an "attribute")
  • PRIMARY KEY = Unique identifier for each row
  • FOREIGN KEY = Links to another table
  • SCHEMA = The structure/blueprint of your database
""")

print("\n📋 SQLite Data Types:")
print("  • INTEGER - Whole numbers (1, 42, -100)")
print("  • REAL - Floating point (3.14, -0.5)")
print("  • TEXT - Strings ('hello', 'world')")
print("  • BLOB - Binary data (images, files)")
print("  • NULL - No value")

---

## 📝 Section 2: Creating Tables and Connections

Let's explore different ways to connect to SQLite and create tables.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: Manual connection management
# ═══════════════════════════════════════════════════════════════════════════════
# ❌ Problem: Connection might not be closed if error occurs

import sqlite3

# Connect to in-memory database (temporary, for learning)
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

# Create a table
cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)')

# Insert data
cur.executemany('INSERT INTO users (name, age) VALUES (?, ?)', 
                [('Ana', 30), ('Bob', 25), ('Cara', 35)])
conn.commit()

# Query data
cur.execute('SELECT * FROM users')
print("All users:", cur.fetchall())

# ⚠️ What's wrong here:
# - Connection stays open if we forget to close
# - No error handling
# - If exception occurs, commit might not happen
# - Must remember: conn.close() at the end!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌱 NOVICE: Basic database setup with try/finally
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: Ensures connection is closed

import sqlite3

conn = None
try:
    # Connect to in-memory database
    conn = sqlite3.connect(':memory:')
    cur = conn.cursor()
    
    # Create table with better schema
    cur.execute('''
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            grade INTEGER CHECK(grade >= 0 AND grade <= 100),
            enrolled_date TEXT DEFAULT CURRENT_DATE
        )
    ''')
    
    # Insert some students
    students = [
        ('Alice', 95),
        ('Bob', 82),
        ('Charlie', 78),
        ('Diana', 91)
    ]
    cur.executemany('INSERT INTO students (name, grade) VALUES (?, ?)', students)
    conn.commit()
    
    # Query and display
    cur.execute('SELECT * FROM students')
    print("📚 Students Table:")
    print("-" * 50)
    for row in cur.fetchall():
        print(f"  ID: {row[0]}, Name: {row[1]}, Grade: {row[2]}")
    
finally:
    if conn:
        conn.close()
        print("\n✅ Connection closed properly")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 INTERMEDIATE: Context manager for automatic cleanup
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Better: 'with' statement handles connection lifecycle

import sqlite3

# Use context manager - connection auto-closes when block ends
with sqlite3.connect(':memory:') as conn:
    # Row factory gives us named columns
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    
    # Create table
    cur.execute('''
        CREATE TABLE products (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            price REAL NOT NULL,
            in_stock INTEGER DEFAULT 0
        )
    ''')
    
    # Insert products
    products = [
        ('Laptop', 999.99, 10),
        ('Mouse', 29.99, 50),
        ('Keyboard', 79.99, 30),
        ('Monitor', 299.99, 15)
    ]
    cur.executemany(
        'INSERT INTO products (name, price, in_stock) VALUES (?, ?, ?)', 
        products
    )
    
    # Query with named columns
    cur.execute('SELECT * FROM products')
    
    print("🛒 Products Table:")
    print("-" * 60)
    print(f"{'ID':<5} {'Name':<15} {'Price':>10} {'In Stock':>10}")
    print("-" * 60)
    
    for row in cur.fetchall():
        # Access by column name thanks to row_factory!
        print(f"{row['id']:<5} {row['name']:<15} ${row['price']:>9.2f} {row['in_stock']:>10}")

# Connection automatically closed here!
print("\n✅ Connection auto-closed by context manager")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ PYTHONIC: Complete database helper class
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Best: Reusable class with proper error handling and logging

import sqlite3
from typing import List, Dict, Any, Optional

class Database:
    """
    A Pythonic database helper class with context manager support.
    
    Usage:
        with Database(':memory:') as db:
            db.execute('CREATE TABLE ...')
            results = db.query('SELECT * FROM ...')
    """
    
    def __init__(self, db_path: str):
        """Initialize with database file path (or ':memory:')."""
        self.db_path = db_path
        self.conn: Optional[sqlite3.Connection] = None
    
    def __enter__(self):
        """Connect when entering context."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.row_factory = sqlite3.Row
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Commit and close when exiting context."""
        if self.conn:
            if exc_type is None:
                self.conn.commit()
            else:
                self.conn.rollback()
            self.conn.close()
    
    def execute(self, sql: str, params: tuple = ()) -> sqlite3.Cursor:
        """Execute a single SQL statement."""
        return self.conn.execute(sql, params)
    
    def executemany(self, sql: str, params_list: List[tuple]) -> sqlite3.Cursor:
        """Execute SQL with multiple parameter sets."""
        return self.conn.executemany(sql, params_list)
    
    def query(self, sql: str, params: tuple = ()) -> List[Dict[str, Any]]:
        """Execute a SELECT and return results as list of dicts."""
        cursor = self.conn.execute(sql, params)
        columns = [desc[0] for desc in cursor.description]
        return [dict(zip(columns, row)) for row in cursor.fetchall()]
    
    def query_one(self, sql: str, params: tuple = ()) -> Optional[Dict[str, Any]]:
        """Execute a SELECT and return first result."""
        results = self.query(sql, params)
        return results[0] if results else None

# ═══════════════════════════════════════════════════════════════════════════════
# 📊 Demonstration
# ═══════════════════════════════════════════════════════════════════════════════

with Database(':memory:') as db:
    # Create table
    db.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            department TEXT,
            salary REAL,
            hire_date TEXT DEFAULT CURRENT_DATE
        )
    ''')
    
    # Insert employees
    db.executemany(
        'INSERT INTO employees (name, department, salary) VALUES (?, ?, ?)',
        [
            ('Alice Johnson', 'Engineering', 95000),
            ('Bob Smith', 'Marketing', 72000),
            ('Carol White', 'Engineering', 88000),
            ('David Brown', 'Sales', 67000),
            ('Eve Davis', 'Engineering', 102000)
        ]
    )
    
    # Query all employees
    employees = db.query('SELECT * FROM employees ORDER BY salary DESC')
    
    print("👥 Employee Database")
    print("=" * 70)
    for emp in employees:
        print(f"  {emp['name']:<20} {emp['department']:<15} ${emp['salary']:,.2f}")
    
    # Query one specific employee
    alice = db.query_one('SELECT * FROM employees WHERE name LIKE ?', ('Alice%',))
    print(f"\n🔍 Found: {alice['name']} - ${alice['salary']:,.2f}")
    
    # Aggregate query
    stats = db.query('''
        SELECT department, COUNT(*) as count, AVG(salary) as avg_salary
        FROM employees
        GROUP BY department
    ''')
    
    print("\n📊 Department Statistics:")
    for s in stats:
        print(f"  {s['department']:<15} {s['count']} employees, avg ${s['avg_salary']:,.2f}")

---

## 🔒 Section 3: SQL Injection Prevention

One of the most critical security concepts in database programming!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚫 BAD PRACTICE: SQL Injection Vulnerability! 
# ═══════════════════════════════════════════════════════════════════════════════
# ❌ NEVER do this in real code!

import sqlite3

# Setup a simple database
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, username TEXT, password TEXT)')
cur.executemany('INSERT INTO users (username, password) VALUES (?, ?)',
                [('admin', 'secret123'), ('user1', 'pass456')])
conn.commit()

# ⚠️ DANGEROUS: String formatting in SQL
def login_VULNERABLE(username, password):
    """THIS IS VULNERABLE - NEVER DO THIS!"""
    # Building SQL with string formatting = disaster!
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"
    print(f"🔴 Query: {query}")
    cur.execute(query)
    return cur.fetchone()

# Normal usage works...
print("Normal login attempt:")
result = login_VULNERABLE('admin', 'secret123')
print(f"Result: {result}")

print()

# But an attacker can inject SQL!
print("🚨 SQL INJECTION ATTACK:")
malicious_input = "' OR '1'='1"
result = login_VULNERABLE(malicious_input, malicious_input)
print(f"Result: {result}")  # Returns admin user without knowing password!

print()
print("⚠️ THE ATTACK WORKS BECAUSE:")
print("   Original: WHERE username = 'X' AND password = 'Y'")
print("   Injected: WHERE username = '' OR '1'='1' AND password = '' OR '1'='1'")
print("   Since '1'='1' is always TRUE, the query returns all users!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✨ SAFE: Parameterized Queries
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ Always use ? placeholders with parameter tuples!

import sqlite3

# Fresh database
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute('CREATE TABLE users (id INTEGER PRIMARY KEY, username TEXT, password TEXT)')
cur.executemany('INSERT INTO users (username, password) VALUES (?, ?)',
                [('admin', 'secret123'), ('user1', 'pass456')])
conn.commit()

# ✅ SAFE: Parameterized query
def login_SAFE(username, password):
    """SAFE - Parameters are properly escaped."""
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    print(f"🟢 Query: {query}")
    print(f"   Params: ({username!r}, {password!r})")
    cur.execute(query, (username, password))
    return cur.fetchone()

# Normal usage
print("Normal login attempt:")
result = login_SAFE('admin', 'secret123')
print(f"✅ Result: {result}")

print()

# Injection attempt fails!
print("🛡️ SQL Injection attempt (blocked):")
malicious_input = "' OR '1'='1"
result = login_SAFE(malicious_input, malicious_input)
print(f"✅ Result: {result}")  # None - attack blocked!

print()
print("🛡️ WHY IT'S SAFE:")
print("   • The ? placeholder treats the entire input as a string literal")
print("   • Special characters like ' are automatically escaped")
print("   • The database sees: WHERE username = '\\' OR \\'1\\'=\\'1'")
print("   • No user with that literal username exists = no results")

---

## 🔄 Section 4: CRUD Operations

CRUD = **C**reate, **R**ead, **U**pdate, **D**elete - the four basic database operations.

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📋 CRUD OPERATIONS: Complete Example
# ═══════════════════════════════════════════════════════════════════════════════

import sqlite3

# Create database and table
conn = sqlite3.connect(':memory:')
conn.row_factory = sqlite3.Row
cur = conn.cursor()

cur.execute('''
    CREATE TABLE tasks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        completed INTEGER DEFAULT 0,
        priority TEXT CHECK(priority IN ('low', 'medium', 'high')) DEFAULT 'medium',
        created_at TEXT DEFAULT CURRENT_TIMESTAMP
    )
''')

print("═" * 70)
print("📝 CRUD OPERATIONS DEMO")
print("═" * 70)

# ═══════════════════════════════════════════════════════════════════════════════
# CREATE - INSERT new records
# ═══════════════════════════════════════════════════════════════════════════════
print("\n➕ CREATE (INSERT):")

# Single insert
cur.execute(
    'INSERT INTO tasks (title, priority) VALUES (?, ?)',
    ('Learn SQL basics', 'high')
)

# Multiple inserts
tasks = [
    ('Practice Python', 'high'),
    ('Review notes', 'medium'),
    ('Coffee break', 'low'),
    ('Build project', 'high')
]
cur.executemany('INSERT INTO tasks (title, priority) VALUES (?, ?)', tasks)
conn.commit()

print(f"   ✅ Inserted {cur.rowcount} tasks")
print(f"   Last ID: {cur.lastrowid}")

# ═══════════════════════════════════════════════════════════════════════════════
# READ - SELECT records
# ═══════════════════════════════════════════════════════════════════════════════
print("\n📖 READ (SELECT):")

# All tasks
cur.execute('SELECT * FROM tasks')
print("   All tasks:")
for row in cur.fetchall():
    status = "✅" if row['completed'] else "⬜"
    print(f"      {status} [{row['priority']}] {row['title']}")

# Filtered query
cur.execute('SELECT * FROM tasks WHERE priority = ?', ('high',))
print("\n   High priority only:")
for row in cur.fetchall():
    print(f"      🔴 {row['title']}")

# ═══════════════════════════════════════════════════════════════════════════════
# UPDATE - Modify records
# ═══════════════════════════════════════════════════════════════════════════════
print("\n✏️ UPDATE:")

# Mark a task complete
cur.execute(
    'UPDATE tasks SET completed = 1 WHERE title = ?',
    ('Learn SQL basics',)
)
conn.commit()
print(f"   ✅ Updated {cur.rowcount} row(s)")

# Change priority
cur.execute(
    'UPDATE tasks SET priority = ? WHERE title LIKE ?',
    ('high', '%project%')
)
conn.commit()

# ═══════════════════════════════════════════════════════════════════════════════
# DELETE - Remove records
# ═══════════════════════════════════════════════════════════════════════════════
print("\n🗑️ DELETE:")

# Delete low priority tasks
cur.execute('DELETE FROM tasks WHERE priority = ?', ('low',))
conn.commit()
print(f"   🗑️ Deleted {cur.rowcount} row(s)")

# Final state
print("\n📋 Final Task List:")
cur.execute('SELECT * FROM tasks ORDER BY completed, priority DESC')
for row in cur.fetchall():
    status = "✅" if row['completed'] else "⬜"
    emoji = {'high': '🔴', 'medium': '🟡', 'low': '🟢'}.get(row['priority'], '⚪')
    print(f"   {status} {emoji} {row['title']}")

conn.close()

---

## 🎯 Section 5: Practice Challenges

Try these exercises to reinforce your SQL skills!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 CHALLENGE 1: Student Grades Database
# ═══════════════════════════════════════════════════════════════════════════════
# Create a database to track student grades

# TODO:
# 1. Create a 'students' table with columns:
#    - id (INTEGER PRIMARY KEY)
#    - name (TEXT NOT NULL)
#    - subject (TEXT)
#    - grade (INTEGER between 0-100)
#
# 2. Insert at least 5 students with different grades
#
# 3. Write queries to find:
#    - All students with grade > 80
#    - Average grade per subject
#    - Student with highest grade
#
# 4. Update a student's grade
#
# 5. Delete students with grade < 60

import sqlite3

# Your code here:
pass  # Replace with your implementation!

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# ✅ SOLUTION: Challenge 1 - Student Grades Database
# ═══════════════════════════════════════════════════════════════════════════════

import sqlite3

with sqlite3.connect(':memory:') as conn:
    conn.row_factory = sqlite3.Row
    cur = conn.cursor()
    
    # 1. Create table
    cur.execute('''
        CREATE TABLE students (
            id INTEGER PRIMARY KEY,
            name TEXT NOT NULL,
            subject TEXT,
            grade INTEGER CHECK(grade >= 0 AND grade <= 100)
        )
    ''')
    
    # 2. Insert students
    students = [
        ('Alice', 'Math', 92),
        ('Bob', 'Math', 78),
        ('Carol', 'Science', 85),
        ('David', 'Science', 55),
        ('Eve', 'Math', 95),
        ('Frank', 'Science', 72),
        ('Grace', 'Math', 88)
    ]
    cur.executemany(
        'INSERT INTO students (name, subject, grade) VALUES (?, ?, ?)',
        students
    )
    
    # 3a. Students with grade > 80
    print("🎯 Students with grade > 80:")
    cur.execute('SELECT name, subject, grade FROM students WHERE grade > 80')
    for row in cur.fetchall():
        print(f"   {row['name']}: {row['grade']} ({row['subject']})")
    
    # 3b. Average grade per subject
    print("\n📊 Average grade by subject:")
    cur.execute('''
        SELECT subject, AVG(grade) as avg_grade, COUNT(*) as count
        FROM students
        GROUP BY subject
    ''')
    for row in cur.fetchall():
        print(f"   {row['subject']}: {row['avg_grade']:.1f} avg ({row['count']} students)")
    
    # 3c. Highest grade
    print("\n🏆 Top student:")
    cur.execute('SELECT name, grade FROM students ORDER BY grade DESC LIMIT 1')
    top = cur.fetchone()
    print(f"   {top['name']} with {top['grade']}%")
    
    # 4. Update a grade
    cur.execute('UPDATE students SET grade = 82 WHERE name = ?', ('Bob',))
    print(f"\n✏️ Updated Bob's grade to 82")
    
    # 5. Delete failing students
    cur.execute('DELETE FROM students WHERE grade < 60')
    print(f"🗑️ Removed {cur.rowcount} student(s) with grade < 60")
    
    # Final roster
    print("\n📋 Final Student Roster:")
    cur.execute('SELECT * FROM students ORDER BY grade DESC')
    for row in cur.fetchall():
        print(f"   {row['name']:<10} {row['subject']:<10} {row['grade']}%")

---

## 📚 Session Recap

### What We Learned Today:

| Topic | Key Concept | SQL Example |
|-------|-------------|-------------|
| **Connections** | Context managers | `with sqlite3.connect() as conn:` |
| **Creating Tables** | Schema definition | `CREATE TABLE ... (columns)` |
| **INSERT** | Add new data | `INSERT INTO ... VALUES (?, ?)` |
| **SELECT** | Query data | `SELECT * FROM ... WHERE ...` |
| **UPDATE** | Modify data | `UPDATE ... SET ... WHERE ...` |
| **DELETE** | Remove data | `DELETE FROM ... WHERE ...` |
| **Security** | Parameterized queries | Use `?` placeholders, NEVER f-strings |

### SQL Query Cheat Sheet:

```sql
-- Create table
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    email TEXT UNIQUE
);

-- Insert
INSERT INTO users (name, email) VALUES (?, ?);

-- Select with conditions
SELECT * FROM users WHERE age > 21 ORDER BY name;

-- Update
UPDATE users SET email = ? WHERE id = ?;

-- Delete
DELETE FROM users WHERE id = ?;

-- Aggregate functions
SELECT COUNT(*), AVG(age), MAX(salary) FROM users;

-- Grouping
SELECT department, AVG(salary) FROM employees GROUP BY department;
```

### The Evolution of Our Code:

| Level | Characteristics |
|-------|-----------------|
| 🚫 **Bad** | String formatting, no error handling |
| 🌱 **Novice** | try/finally, basic parameterized queries |
| 🔧 **Intermediate** | Context manager, row_factory |
| ✨ **Pythonic** | Reusable Database class, type hints |

### 🔒 Security Golden Rules:

1. **ALWAYS** use parameterized queries (`?` placeholders)
2. **NEVER** use f-strings or `.format()` with user input
3. **ALWAYS** validate input before database operations
4. **USE** context managers to ensure proper cleanup

### 🎯 Next Steps:

- Practice with more complex queries (JOINs, subqueries)
- Learn about database normalization
- Explore SQLAlchemy ORM
- Build a project with persistent data storage!